In [1]:
import pandas as pd

In [2]:
import datetime as datetime

In [3]:
access_path = '../../data/processed/'
bal_file = 'bal_btw_rebal.dat'
# bal_file = 'balance_rebaldate_0801_all.dat'

#### 매일의 잔고(detail 테이블) INSERT

In [4]:
df_bal = pd.read_csv(access_path+bal_file, parse_dates=['date'])

In [5]:
df_bal

,date,itemcode,userid,name,asset_class,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,original
0,2021-09-02,A114260,A001,안정추구형소규모,Fixed Income,KODEX 국고채3년,2.0,58140.0,116280.0,58060.0,116120.0,0.162425,NaN,NaN
1,2021-09-02,A122260,A001,안정추구형소규모,Fixed Income,KOSEF 통안채1년,1.0,101265.0,101265.0,101295.0,101295.0,0.141688,NaN,NaN
2,2021-09-02,A130730,A001,안정추구형소규모,Fixed Income,KOSEF 단기자금,2.0,101190.0,202380.0,101205.0,202410.0,0.283125,NaN,NaN
3,2021-09-02,A196230,A001,안정추구형소규모,Fixed Income,KBSTAR 단기통안채,1.0,104535.0,104535.0,104560.0,104560.0,0.146255,NaN,NaN
4,2021-09-02,A273130,A001,안정추구형소규모,Fixed Income,KODEX 종합채권(AA-이상)액티브,1.0,109870.0,109870.0,109825.0,109825.0,0.153620,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,2021-09-27,A278540,A009,적극투자형대규모,Equity,KODEX MSCI Korea TR,24.0,13490.0,323760.0,13180.0,316320.0,0.245707,NaN,NaN
1868,2021-09-27,A102780,A009,적극투자형대규모,Equity,KODEX 삼성그룹,17.0,10555.0,179435.0,10325.0,175525.0,0.136342,NaN,NaN
1869,2021-09-27,A069500,A009,적극투자형대규모,Equity,KODEX 200,3.0,41985.0,125955.0,41100.0,123300.0,0.095776,NaN,NaN
1870,2021-09-27,A114260,A009,적극투자형대규모,Fixed Income,KODEX 국고채3년,1.0,58140.0,58140.0,57835.0,57835.0,0.044924,NaN,NaN


In [6]:
df_bal.date = df_bal['date'].dt.strftime('%#m/%#d/%Y 4:%M:%S PM')
# df_bal.date = df_bal['date'].dt.strftime('%-m/%-d/%Y 4:%M:%S PM')

In [7]:
df_bal.date

0        9/2/2021 4:00:00 PM
1        9/2/2021 4:00:00 PM
2        9/2/2021 4:00:00 PM
3        9/2/2021 4:00:00 PM
4        9/2/2021 4:00:00 PM
                ...         
1867    9/27/2021 4:00:00 PM
1868    9/27/2021 4:00:00 PM
1869    9/27/2021 4:00:00 PM
1870    9/27/2021 4:00:00 PM
1871    9/27/2021 4:00:00 PM
Name: date, Length: 1872, dtype: object

In [8]:
from sqlalchemy import create_engine

# Defining our connection variables
username = 'kdyjpzwsyqrqpb' # replace with your username
password = '6b610da9417e361c2af3be1fd0c179061b5b258f0e6f7ff9e1da464ba221e46c' # replace with your password
ipaddress = 'ec2-35-174-35-242.compute-1.amazonaws.com' # change this to your db’s IP address is if not Panoply
port = 5432 # this is the standard port for Postgres, but change it to your port if needed
dbname = 'd2ke5vkurjkusr' # change this to...you guessed it...the name of your db

# A long string that contains the necessary Postgres login information
postgres_str = f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
          
# Create the connection
cnx = create_engine(postgres_str)

In [9]:
table_nm = 'detail'
df_bal.to_sql(table_nm, con=cnx, index=False, if_exists='append')

### 자산군별 비중(general 테이블) INSERT

In [10]:
df_bal = pd.read_csv(access_path+bal_file, parse_dates=['date'])

In [11]:
df_bal_gby = df_bal.loc[:, ['date', 'userid', 'value', 'asset_class']].sort_values(
    ['date', 'userid'],
    ascending=True).groupby(['date', 'userid', 'asset_class'
                             ]).sum().reset_index(drop=False)

In [12]:
df_bal_gby = df_bal_gby.assign(wt=df_bal_gby.groupby(['date', 'userid'])['value'].transform(lambda x: x/x.sum()))

In [13]:
df_bal_gby.date = df_bal_gby['date'].dt.strftime('%#m/%#d/%Y 4:%M:%S PM')
# df_bal_gby.date = df_bal_gby['date'].dt.strftime('%-m/%-d/%Y 4:%M:%S PM')

In [14]:
df_bal_gby

,date,userid,asset_class,value,wt
0,9/2/2021 4:00:00 PM,A001,Alternative,12245.0,0.017128
1,9/2/2021 4:00:00 PM,A001,Cash,6210.0,0.008686
2,9/2/2021 4:00:00 PM,A001,Equity,62250.0,0.087073
3,9/2/2021 4:00:00 PM,A001,Fixed Income,634210.0,0.887112
4,9/2/2021 4:00:00 PM,A002,Alternative,24490.0,0.024369
...,...,...,...,...,...
931,9/27/2021 4:00:00 PM,A008,Fixed Income,57835.0,0.058274
932,9/27/2021 4:00:00 PM,A009,Alternative,331290.0,0.257336
933,9/27/2021 4:00:00 PM,A009,Cash,2785.0,0.002163
934,9/27/2021 4:00:00 PM,A009,Equity,895475.0,0.695577


In [15]:
table_nm = 'general'
df_bal_gby.to_sql(table_nm, con=cnx, index=False, if_exists='append')